In [1]:
import csv
import pandas as pd
import pickle
import re
from sklearn.model_selection import train_test_split
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import numpy as np
import os.path
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [17]:
data = pd.read_excel("Rest_Mex_2022.xlsx")
data.head(10)

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel
5,El peor huevos Benedict jamás,Hoy teníamos el desayuno por segunda vez en po...,1,Restaurant
6,Definitivamente no volvería a hospedarme en Sa...,"El hotel en si no es malo, pero mi experiencia...",1,Hotel
7,Terrible,No estoy seguro de por qué este restaurante ti...,1,Restaurant
8,"Bebidas ADULTERADAS, FATAL mi experiencia!!!",Llegué a este hotel “por desgracia” mi reserva...,1,Hotel
9,Hotel ha ido cuesta abajo,Hemos estado viniendo a Villa la Estancia dura...,1,Hotel


In [18]:
data['Opinion'][7693]

'Aunque me parece algo hipócrita el arte de Rivera (todo lo ve desde fuera desde una zona segura), debo reconocer que tiene algunas obras que me agradan. Este museo es interesante por el simple hecho de conocer el primer hogar que fuera de Rivera. Bastante fancy. Está chido si te gusta ver cosas antiguas.'

In [19]:
myTest = data.head(10000)
print(myTest)

                                      Title  \
0                              Pésimo lugar   
1                 No vayas a lugar de Eddie   
2              Mala relación calidad-precio   
3          Minusválido? ¡No te alojes aquí!   
4     Es una porqueria no pierdan su tiempo   
...                                     ...   
9995                               Vacacion   
9996                 Excelente y muy barato   
9997                      Mi lugar favorito   
9998                     Joysquad excelente   
9999               El hotel y sus empleados   

                                                Opinion  Polarity  Attraction  
0     Piensen dos veces antes de ir a este hotel, te...         1       Hotel  
1     Cuatro de nosotros fuimos recientemente a Eddi...         1  Restaurant  
2     seguiré corta y simple: limpieza\n- bad. Tengo...         1       Hotel  
3     Al reservar un hotel con multipropiedad Mayan ...         1       Hotel  
4     No pierdan su tiempo ni diner

# Normalizacion

In [20]:
nlp = spacy.load("es_core_news_sm")

In [21]:
def tokenizar(doc):
    if(type(doc) != type("")):
         print(doc)
         return ""
    # nlp = spacy.load("es_core_news_sm") # español
    
    tkDoc = nlp(doc)
    res = ""
    for token in tkDoc:
        if(not(token.is_stop)): # quitando stop words
                res+=token.lemma_
                res+=" "

    return res
# print(newsgroups_train.data[0])

In [22]:
myTest['Opinion'] = myTest['Opinion'].apply(tokenizar)

nan


C:\Users\sergi\AppData\Local\Temp\ipykernel_21960\77379802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myTest['Opinion'] = myTest['Opinion'].apply(tokenizar)


In [23]:
myTest

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"piensir hotel , molestar , hijo año traer infl...",1,Hotel
1,No vayas a lugar de Eddie,recientemente Eddie's Place . servicio terribl...,1,Restaurant
2,Mala relación calidad-precio,seguir corto simple : limpieza \n - bad . insi...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,reservar hotel multipropiedad Mayan grupo deci...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"perder tiempo dinero , venir tener boda , comp...",1,Hotel
...,...,...,...,...
9995,Vacacion,"excelente lugar vacacionar , servicio calidad ...",5,Hotel
9996,Excelente y muy barato,"carne , tortilla recién hecho , queso fundido ...",5,Restaurant
9997,Mi lugar favorito,", seguido , comer delicioso atender maravilla ...",5,Restaurant
9998,Joysquad excelente,Lisbhet Torre Punta excelente servicio amable ...,5,Hotel


In [24]:
myTest['Polarity'].value_counts()

Polarity
4    5878
3    2121
2     730
5     724
1     547
Name: count, dtype: int64

In [25]:
result = myTest[myTest['Opinion']=="error"]

In [26]:
result

,Title,Opinion,Polarity,Attraction


In [27]:
myTest['Title'] = myTest['Title'].apply(tokenizar)

2016-12-01 00:00:00
2015-05-01 00:00:00


C:\Users\sergi\AppData\Local\Temp\ipykernel_21960\745785095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myTest['Title'] = myTest['Title'].apply(tokenizar)


In [28]:
myTest

,Title,Opinion,Polarity,Attraction
0,pésimo lugar,"piensir hotel , molestar , hijo año traer infl...",1,Hotel
1,vayas lugar Eddie,recientemente Eddie's Place . servicio terribl...,1,Restaurant
2,malo relación calidad-precio,seguir corto simple : limpieza \n - bad . insi...,1,Hotel
3,minusválido ? ¡ alojes !,reservar hotel multipropiedad Mayan grupo deci...,1,Hotel
4,porquerio perder tiempo,"perder tiempo dinero , venir tener boda , comp...",1,Hotel
...,...,...,...,...
9995,Vacacion,"excelente lugar vacacionar , servicio calidad ...",5,Hotel
9996,excelente barato,"carne , tortilla recién hecho , queso fundido ...",5,Restaurant
9997,lugar favorito,", seguido , comer delicioso atender maravilla ...",5,Restaurant
9998,Joysquad excelente,Lisbhet Torre Punta excelente servicio amable ...,5,Hotel


# Splitting

In [29]:
train, test = train_test_split(myTest, test_size=0.2)

In [30]:
train

,Title,Opinion,Polarity,Attraction
9659,Marival distinct,"agradable visita , servicio mesero servicio ge...",5,Hotel
3234,fan …,"seguidor mercado tipo , opción artesanía dulce...",3,Attractive
7293,selección,menú tamaño opción buffet . Genial desayuno qu...,4,Restaurant
3680,Estancia 3 semana,tener problema cambiar dinero . demorar transa...,4,Hotel
2936,Invaluable visto,"calificar mirador ciudad , lugar deber dejar v...",3,Attractive
...,...,...,...,...
5932,,lugar ! serviciir rápido opción corte carne . ...,4,Restaurant
3403,"Restaurante desayuno hielo agradable , fácil l...","encantar gofr esposa encantar huevo . , impuls...",4,Restaurant
5170,"excelente trato personal huesped , especial ar...","playa bonito , trato personal excelente gana r...",4,Hotel
2033,Mala atencion,hotel habitación cómodo general limpieza . emp...,3,Hotel


In [31]:
test

,Title,Opinion,Polarity,Attraction
8772,vista ciudad,mirador podrás admirar belleza ciudad Guanajua...,4,Attractive
2976,Artesanías,"quedar , interesar artesania garnacha . atract...",3,Attractive
7633,- pescado italiano,tour canal buscar Gators . tener parmesana pol...,4,Restaurant
6925,variedad agua,"pesca decidir tomar camarón azul Vallarta , op...",4,Restaurant
9489,comida Tino's,tiempo Tino vacación Vallarta . tener ceviche ...,5,Restaurant
...,...,...,...,...
4700,mexicano .,Grupo 10 comimos celebrar cumpleaños decepcion...,4,Restaurant
5259,RELAJANTE FELIZ,"problema reservacion , . gente super servicial...",4,Hotel
3415,instalación,Gozan servicio instalación ubicado centro come...,4,Hotel
1531,lugar comer Mariscos,"luegar agradable vista lagunita , servicio ali...",3,Restaurant


# Text Representations

In [32]:
vectorizador_binario = CountVectorizer(binary=True, token_pattern= r'(?u)\w\w+|\w\w+\n|\.')
vectorizador_frecuencia = CountVectorizer(token_pattern= r'(?u)\w\w+|\w\w+\n|\.')
vectorizador_tfidf = TfidfVectorizer()

In [33]:
train_vectorizado = vectorizador_tfidf.fit_transform(train['Opinion'])

In [34]:
test_vectorizado = vectorizador_tfidf.transform(test['Opinion'])

In [35]:
print(vectorizador_tfidf.get_feature_names_out())
print(train_vectorizado.toarray())
print(len(train_vectorizado.toarray()[0]))
print(len(vectorizador_tfidf.get_feature_names_out()))

['00' '000' '00am' ... 'útil' 'útlimo' 'ĺlegar']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
23209
23209


# Using Sentiment lexicons

In [36]:
def load_sel():
	#~ global lexicon_sel
	lexicon_sel = {}
	input_file = open('SEL_full.txt', 'r', encoding="utf-8")
	for line in input_file:
		#Las líneas del lexicon tienen el siguiente formato:
		#abundancia	0	0	50	50	0.83	Alegría
		
		palabras = line.split("\t")
		palabras[6]= re.sub('\n', '', palabras[6])
		pair = (palabras[6], palabras[5])
		if lexicon_sel:
			if palabras[0] not in lexicon_sel:
				lista = [pair]
				lexicon_sel[palabras[0]] = lista
			else:
				lexicon_sel[palabras[0]].append (pair)
		else:
			lista = [pair]
			lexicon_sel[palabras[0]] = lista
	input_file.close()
	del lexicon_sel['Palabra']; #Esta llave se inserta porque es parte del encabezado del diccionario, por lo que se requiere eliminar
	#Estructura resultante
		#'hastiar': [('Enojo\n', '0.629'), ('Repulsi\xf3n\n', '0.596')]
	return lexicon_sel

In [37]:
if (os.path.exists('lexicon_sel.pkl')):
    lexicon_sel_file = open ('lexicon_sel.pkl','rb')
    lexicon_sel = pickle.load(lexicon_sel_file)
else:
    lexicon_sel = load_sel()
    lexicon_sel_file = open ('lexicon_sel.pkl','wb')
    pickle.dump(lexicon_sel, lexicon_sel_file)
    lexicon_sel_file.close()

In [38]:
print(lexicon_sel['gustar'])

[('Alegría', '0.596')]


In [39]:
def getSELFeatures(cadenas, lexicon_sel):
	#'hastiar': [('Enojo\n', '0.629'), ('Repulsi\xf3n\n', '0.596')]
	features = []
	for cadena in cadenas:
		valor_alegria = 0.0
		valor_enojo = 0.0
		valor_miedo = 0.0
		valor_repulsion = 0.0
		valor_sorpresa = 0.0
		valor_tristeza = 0.0
		cadena_palabras = re.split('\s+', cadena)
		dic = {}
		for palabra in cadena_palabras:
			if palabra in lexicon_sel:
				caracteristicas = lexicon_sel[palabra]
				for emocion, valor in caracteristicas:
					if emocion == 'AlegrÃ\xada':
						valor_alegria = valor_alegria + float(valor)
					elif emocion == 'Tristeza':
						valor_tristeza = valor_tristeza + float(valor)
					elif emocion == 'Enojo':
						valor_enojo = valor_enojo + float(valor)
					elif emocion == 'Repulsión':
						valor_repulsion = valor_repulsion + float(valor)
					elif emocion == 'Miedo':
						valor_miedo = valor_miedo + float(valor)
					elif emocion == 'Sorpresa':
						valor_sorpresa = valor_sorpresa + float(valor)
		dic['__txt__'] = cadena_palabras
		dic['__alegria__'] = valor_alegria
		dic['__tristeza__'] = valor_tristeza
		dic['__enojo__'] = valor_enojo
		dic['__repulsion__'] = valor_repulsion
		dic['__miedo__'] = valor_miedo
		dic['__sorpresa__'] = valor_sorpresa
		
		#Esto es para los valores acumulados del mapeo a positivo (alegría + sorpresa) y negativo (enojo + miedo + repulsión + tristeza)
		dic['acumuladopositivo'] = dic['__alegria__'] + dic['__sorpresa__']
		dic['acumuladonegative'] = dic['__enojo__'] + dic['__miedo__'] + dic['__repulsion__'] + dic['__tristeza__']
		
		features.append (dic)
	
	
	return features

In [40]:
polaridad = getSELFeatures(train['Opinion'], lexicon_sel)
print("TEXTO")
print(train['Opinion'][7693])
print("POLARIDAD")
print (polaridad[0])

TEXTO
hipócrita arte Rivera ( ver zona seguro ) , deber reconocer obra agradir . museo interesante simple hogar Rivera . fancy . chido gustar cosa antiguo . 
POLARIDAD
{'__txt__': ['agradable', 'visita', ',', 'servicio', 'mesero', 'servicio', 'general', 'increíble', ',', 'comida', 'buenísimo', 'Oscar', 'Carlos', 'Pineda', 'amable', 'servicio', 'valer', 'pena', 'lugar', '🤩', 'lugar', 'hermoso', 'encantar', 'valer', 'pena', 'venir', ''], '__alegria__': 0.0, '__tristeza__': 1.39, '__enojo__': 0.0, '__repulsion__': 0.0, '__miedo__': 0.66, '__sorpresa__': 0.0, 'acumuladopositivo': 0.0, 'acumuladonegative': 2.05}


# Training ML models

In [41]:
x_train = train_vectorizado
y_train = train['Polarity']
x_test = test_vectorizado
y_test = test['Polarity']

In [42]:
clf_polarity = LogisticRegression(max_iter=10000)
clf_polarity.fit(x_train,y_train)
y_pred = clf_polarity.predict(x_test)

In [43]:
clf_bayes = MultinomialNB()
clf_bayes.fit(x_train, y_train)
y_pred = clf_bayes.predict(x_test)

### Lematizacion, stop words. Binarizado. Logistic Regression:

In [44]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Lematizacion, stop words. Frecuencia. Logistic Regression:

In [45]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Lematizacion, stop words. Tf-idf. Logistic Regression:

In [46]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Lematizacion, stop words. Tf-idf. Multinomial Naive Bayes

In [47]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
